In [272]:
import pandas as pd
import numpy as np

import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report

import time

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn import tree
from sklearn.neural_network import MLPClassifier

In [273]:
df = pd.read_csv("data/heart.csv")
df

age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  target  
0        0   0     1       1  
1        0   0     2       1  
2        2   0     2       1  
3        2   0     2       1  
4        2   0     2       1  
..     ...  ..   ...     ...  
298      1   0     3       0  
299      1   0     3       0  
300      1   2     3       0  
301      1   1     3       0  
302      1   1     2       0  

[303 rows x 14 columns]

Fazendo uma exploração inicial dos atributos target, temos

In [274]:
#Analisando primeiro os problemas de coração por idade
df.groupby('target')['age'].count()

target
0    138
1    165
Name: age, dtype: int64

In [275]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [276]:
df.describe()

age         sex          cp    trestbps        chol         fbs  \
count  303.000000  303.000000  303.000000  303.000000  303.000000  303.000000   
mean    54.366337    0.683168    0.966997  131.623762  246.264026    0.148515   
std      9.082101    0.466011    1.032052   17.538143   51.830751    0.356198   
min     29.000000    0.000000    0.000000   94.000000  126.000000    0.000000   
25%     47.500000    0.000000    0.000000  120.000000  211.000000    0.000000   
50%     55.000000    1.000000    1.000000  130.000000  240.000000    0.000000   
75%     61.000000    1.000000    2.000000  140.000000  274.500000    0.000000   
max     77.000000    1.000000    3.000000  200.000000  564.000000    1.000000   

          restecg     thalach       exang     oldpeak       slope          ca  \
count  303.000000  303.000000  303.000000  303.000000  303.000000  303.000000   
mean     0.528053  149.646865    0.326733    1.039604    1.399340    0.729373   
std      0.525860   22.905161    0.469794    1.161075    0.616226    1.022606   
min      0.000000   71.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000  133.500000    0.000000    0.000000    1.000000    0.000000   
50%      1.000000  153.000000    0.000000    0.800000    1.000000    0.000000   
75%      1.000000  166.000000    1.000000    1.600000    2.000000    1.000000   
max      2.000000  202.000000    1.000000    6.200000    2.000000    4.000000   

             thal      target  
count  303.000000  303.000000  
mean     2.313531    0.544554  
std      0.612277    0.498835  
min      0.000000    0.000000  
25%      2.000000    0.000000  
50%      2.000000    1.000000  
75%      3.000000    1.000000  
max      3.000000    1.000000

Como os targets estão bem distribuidos, podemos fazer uma análise inicial e verificarmos as correlações naturalmente

In [277]:
df.corr()

age       sex        cp  trestbps      chol       fbs  \
age       1.000000 -0.098447 -0.068653  0.279351  0.213678  0.121308   
sex      -0.098447  1.000000 -0.049353 -0.056769 -0.197912  0.045032   
cp       -0.068653 -0.049353  1.000000  0.047608 -0.076904  0.094444   
trestbps  0.279351 -0.056769  0.047608  1.000000  0.123174  0.177531   
chol      0.213678 -0.197912 -0.076904  0.123174  1.000000  0.013294   
fbs       0.121308  0.045032  0.094444  0.177531  0.013294  1.000000   
restecg  -0.116211 -0.058196  0.044421 -0.114103 -0.151040 -0.084189   
thalach  -0.398522 -0.044020  0.295762 -0.046698 -0.009940 -0.008567   
exang     0.096801  0.141664 -0.394280  0.067616  0.067023  0.025665   
oldpeak   0.210013  0.096093 -0.149230  0.193216  0.053952  0.005747   
slope    -0.168814 -0.030711  0.119717 -0.121475 -0.004038 -0.059894   
ca        0.276326  0.118261 -0.181053  0.101389  0.070511  0.137979   
thal      0.068001  0.210041 -0.161736  0.062210  0.098803 -0.032019   
target   -0.225439 -0.280937  0.433798 -0.144931 -0.085239 -0.028046   

           restecg   thalach     exang   oldpeak     slope        ca  \
age      -0.116211 -0.398522  0.096801  0.210013 -0.168814  0.276326   
sex      -0.058196 -0.044020  0.141664  0.096093 -0.030711  0.118261   
cp        0.044421  0.295762 -0.394280 -0.149230  0.119717 -0.181053   
trestbps -0.114103 -0.046698  0.067616  0.193216 -0.121475  0.101389   
chol     -0.151040 -0.009940  0.067023  0.053952 -0.004038  0.070511   
fbs      -0.084189 -0.008567  0.025665  0.005747 -0.059894  0.137979   
restecg   1.000000  0.044123 -0.070733 -0.058770  0.093045 -0.072042   
thalach   0.044123  1.000000 -0.378812 -0.344187  0.386784 -0.213177   
exang    -0.070733 -0.378812  1.000000  0.288223 -0.257748  0.115739   
oldpeak  -0.058770 -0.344187  0.288223  1.000000 -0.577537  0.222682   
slope     0.093045  0.386784 -0.257748 -0.577537  1.000000 -0.080155   
ca       -0.072042 -0.213177  0.115739  0.222682 -0.080155  1.000000   
thal     -0.011981 -0.096439  0.206754  0.210244 -0.104764  0.151832   
target    0.137230  0.421741 -0.436757 -0.430696  0.345877 -0.391724   

              thal    target  
age       0.068001 -0.225439  
sex       0.210041 -0.280937  
cp       -0.161736  0.433798  
trestbps  0.062210 -0.144931  
chol      0.098803 -0.085239  
fbs      -0.032019 -0.028046  
restecg  -0.011981  0.137230  
thalach  -0.096439  0.421741  
exang     0.206754 -0.436757  
oldpeak   0.210244 -0.430696  
slope    -0.104764  0.345877  
ca        0.151832 -0.391724  
thal      1.000000 -0.344029  
target   -0.344029  1.000000

# Primeiras Predições

Como já possuímos bons atributos correlacionados, vamos tentar criar um modelo relevante com eles

In [278]:
def classifier(X_train,X_test,Y_train,Y_test):
    #Aplicando KNN
    print(" KNN ")
    ini = time.time() #Inicio do calculo do tempo

    knn=KNeighborsClassifier()
    knn.fit(X_train,Y_train)

    end = time.time()
    knn_time = end - ini # Fim do calculo do tempo

    print(classification_report(Y_test,knn.predict(X_test)))
    print()
    
    #Aplicando Naive Bayes
    print("Naive Bayes")
    ini = time.time() #Inicio do calculo do tempo

    nb=BernoulliNB()
    nb.fit(X_train,Y_train)

    end = time.time()
    nb_time = end - ini # Fim do calculo do tempo

    print(classification_report(Y_test,nb.predict(X_test)))
    print()
    
    #Aplicando Arvore de Decisao
    print("Árvore de Decisão")
    ini = time.time() #Inicio do calculo do tempo

    dtc=tree.DecisionTreeClassifier()
    dtc.fit(X_train,Y_train)

    end = time.time()
    dtc_time = end - ini # Fim do calculo do tempo

    print(classification_report(Y_test,dtc.predict(X_test)))

    #Aplicando MLP
    print("MLP")
    ini = time.time() #Inicio do calculo do tempo

    mlp = MLPClassifier(alpha=1, max_iter=1000)
    mlp.fit(X_train, Y_train)

    end = time.time()
    mlp_time = end - ini # Fim do calculo do tempo

    print(classification_report(Y_test,mlp.predict(X_test)))

    return knn, nb, dtc, mlp

def runModels(relevantFeatures):
    X_train,X_test,Y_train,Y_test=train_test_split(
        df[relevantFeatures], df['target'], test_size=0.2, random_state=0
    )

    knn, nb, dtc, mlp = classifier(X_train,X_test,Y_train,Y_test)

In [279]:
runModels(["cp", "thalach", "exang", "oldpeak", "ca"])

 KNN 
              precision    recall  f1-score   support

           0       0.75      0.78      0.76        27
           1       0.82      0.79      0.81        34

    accuracy                           0.79        61
   macro avg       0.78      0.79      0.78        61
weighted avg       0.79      0.79      0.79        61


Naive Bayes
              precision    recall  f1-score   support

           0       0.82      0.67      0.73        27
           1       0.77      0.88      0.82        34

    accuracy                           0.79        61
   macro avg       0.79      0.77      0.78        61
weighted avg       0.79      0.79      0.78        61


Árvore de Decisão
              precision    recall  f1-score   support

           0       0.76      0.70      0.73        27
           1       0.78      0.82      0.80        34

    accuracy                           0.77        61
   macro avg       0.77      0.76      0.77        61
weighted avg       0.77      0.77   

# Binarização e Arredondamento

Com uma simples análise das features e boas correlações, pudemos obter um modelo razoávelmente aceitável sem muitas manipulações. Vamos tentar agora usar alguma estratégia de agrupamento para melhorarmos o modelo criado.

In [280]:
def binarize(row, attr):
    if(int(row[attr]) >= 1):
        return 1
    return 0

def around(row, attr):    
    return int(np.floor(row[attr]))

In [281]:
df["cp bin"] = df.apply (lambda row: binarize(row, "cp"), axis=1)
df["oldpeak around"] = df.apply (lambda row: around(row, "oldpeak"), axis=1)

#mantém target na última coluna
df["target"] = df.pop("target")

df.corr()

age       sex        cp  trestbps      chol       fbs  \
age             1.000000 -0.098447 -0.068653  0.279351  0.213678  0.121308   
sex            -0.098447  1.000000 -0.049353 -0.056769 -0.197912  0.045032   
cp             -0.068653 -0.049353  1.000000  0.047608 -0.076904  0.094444   
trestbps        0.279351 -0.056769  0.047608  1.000000  0.123174  0.177531   
chol            0.213678 -0.197912 -0.076904  0.123174  1.000000  0.013294   
fbs             0.121308  0.045032  0.094444  0.177531  0.013294  1.000000   
restecg        -0.116211 -0.058196  0.044421 -0.114103 -0.151040 -0.084189   
thalach        -0.398522 -0.044020  0.295762 -0.046698 -0.009940 -0.008567   
exang           0.096801  0.141664 -0.394280  0.067616  0.067023  0.025665   
oldpeak         0.210013  0.096093 -0.149230  0.193216  0.053952  0.005747   
slope          -0.168814 -0.030711  0.119717 -0.121475 -0.004038 -0.059894   
ca              0.276326  0.118261 -0.181053  0.101389  0.070511  0.137979   
thal            0.068001  0.210041 -0.161736  0.062210  0.098803 -0.032019   
cp bin         -0.138253 -0.089621  0.887256 -0.021447 -0.070684  0.060190   
oldpeak around  0.178396  0.089604 -0.180783  0.190276  0.035479  0.022088   
target         -0.225439 -0.280937  0.433798 -0.144931 -0.085239 -0.028046   

                 restecg   thalach     exang   oldpeak     slope        ca  \
age            -0.116211 -0.398522  0.096801  0.210013 -0.168814  0.276326   
sex            -0.058196 -0.044020  0.141664  0.096093 -0.030711  0.118261   
cp              0.044421  0.295762 -0.394280 -0.149230  0.119717 -0.181053   
trestbps       -0.114103 -0.046698  0.067616  0.193216 -0.121475  0.101389   
chol           -0.151040 -0.009940  0.067023  0.053952 -0.004038  0.070511   
fbs            -0.084189 -0.008567  0.025665  0.005747 -0.059894  0.137979   
restecg         1.000000  0.044123 -0.070733 -0.058770  0.093045 -0.072042   
thalach         0.044123  1.000000 -0.378812 -0.344187  0.386784 -0.213177   
exang          -0.070733 -0.378812  1.000000  0.288223 -0.257748  0.115739   
oldpeak        -0.058770 -0.344187  0.288223  1.000000 -0.577537  0.222682   
slope           0.093045  0.386784 -0.257748 -0.577537  1.000000 -0.080155   
ca             -0.072042 -0.213177  0.115739  0.222682 -0.080155  1.000000   
thal           -0.011981 -0.096439  0.206754  0.210244 -0.104764  0.151832   
cp bin          0.094591  0.376561 -0.469061 -0.280243  0.216056 -0.218226   
oldpeak around -0.055906 -0.327627  0.271144  0.966965 -0.555175  0.232167   
target          0.137230  0.421741 -0.436757 -0.430696  0.345877 -0.391724   

                    thal    cp bin  oldpeak around    target  
age             0.068001 -0.138253        0.178396 -0.225439  
sex             0.210041 -0.089621        0.089604 -0.280937  
cp             -0.161736  0.887256       -0.180783  0.433798  
trestbps        0.062210 -0.021447        0.190276 -0.144931  
chol            0.098803 -0.070684        0.035479 -0.085239  
fbs            -0.032019  0.060190        0.022088 -0.028046  
restecg        -0.011981  0.094591       -0.055906  0.137230  
thalach        -0.096439  0.376561       -0.327627  0.421741  
exang           0.206754 -0.469061        0.271144 -0.436757  
oldpeak         0.210244 -0.280243        0.966965 -0.430696  
slope          -0.104764  0.216056       -0.555175  0.345877  
ca              0.151832 -0.218226        0.232167 -0.391724  
thal            1.000000 -0.207277        0.198263 -0.344029  
cp bin         -0.207277  1.000000       -0.293763  0.516015  
oldpeak around  0.198263 -0.293763        1.000000 -0.423572  
target         -0.344029  0.516015       -0.423572  1.000000

In [285]:
runModels(["cp bin", "thalach", "exang", "oldpeak"])

 KNN 
              precision    recall  f1-score   support

           0       0.75      0.67      0.71        27
           1       0.76      0.82      0.79        34

    accuracy                           0.75        61
   macro avg       0.75      0.75      0.75        61
weighted avg       0.75      0.75      0.75        61


Naive Bayes
              precision    recall  f1-score   support

           0       0.78      0.78      0.78        27
           1       0.82      0.82      0.82        34

    accuracy                           0.80        61
   macro avg       0.80      0.80      0.80        61
weighted avg       0.80      0.80      0.80        61


Árvore de Decisão
              precision    recall  f1-score   support

           0       0.70      0.59      0.64        27
           1       0.71      0.79      0.75        34

    accuracy                           0.70        61
   macro avg       0.70      0.69      0.69        61
weighted avg       0.70      0.70   

Utilizando então de técnicas de arredondamento e binarização para tentar obter melhores resultados, podemos observar que o MLP melhorou bastante suas previsões, apesar dos outros métodos manterem na mesma casa entre 70 e 80% de precisão para os targets.